### Importing necessary libraries

In [7]:
import ssl
import pandas as pd
from urllib.request import urlopen
import yfinance as yf

#Disable warnings
import warnings
warnings.filterwarnings("ignore")

### Importing constituents of the SP500 index

In [43]:
# Disabling SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

# Fetch S&P 500 constituents from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
spx_df = pd.read_html(urlopen(url))[0]
spx_df.index = spx_df['Symbol']
spx_df = spx_df.drop(columns=['Symbol', 'CIK', 'Founded'])

#changing the symbol containing a dot to a dash
spx_df.index = spx_df.index.str.replace('.', '-')

In [46]:
spx_df.head()

,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added
Symbol,,,,,
MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04
AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26
ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04
ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31
ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06


### Listing Best and Worst performing stocks of the day

**Best and Worst performers of the day**

In [47]:
today = pd.Timestamp.today()
today_str = today.strftime('%Y-%m-%d')

#if today is a weekend, we need to get the last day of the week 
#getting the close price of the last day
if(today.weekday() == 6):
    today = today - pd.Timedelta(days=2)
    last_day = today - pd.Timedelta(days=1)
elif(today.weekday() == 5):
    today = today - pd.Timedelta(days=1)
    last_day = today - pd.Timedelta(days=1)
else:
    last_day = today - pd.Timedelta(days=1)

In [48]:
#getting the return of each stock in the S&P 500
SPX_returns = pd.DataFrame(index = spx_df.index, columns=['Return', 'Security', 'Sector', 'Industry'])
SPX_returns['Security'] = spx_df['Security']
SPX_returns['Sector'] = spx_df['GICS Sector']
SPX_returns['Industry'] = spx_df['GICS Sub-Industry']
for ticker in spx_df.index:
    try:
        stock = yf.Ticker(ticker)
        stock_data = stock.history(start=last_day, end=today)
        stock_return = (stock_data['Close'][-1] - stock_data['Close'][0])/stock_data['Close'][0]
        SPX_returns.loc[ticker, 'Return' ] = stock_return
    except:
        print('Error getting data for ' + ticker)

In [53]:
#listing the top 10 stocks with the highest return
SPX_returns = SPX_returns.sort_values(by='Return', ascending=False)
print('Top 10 stocks with the highest return in the S&P 500 of ', today_str, ':', sep='')
SPX_returns.head(10)

Top 10 stocks with the highest return in the S&P 500 of 2024-02-10:


,Return,Security,Sector,Industry
Symbol,,,,
AMAT,0.068722,Applied Materials,Information Technology,Semiconductor Materials & Equipment
FSLR,0.058183,First Solar,Information Technology,Semiconductors
LRCX,0.05463,Lam Research,Information Technology,Semiconductor Materials & Equipment
KLAC,0.050623,KLA Corporation,Information Technology,Semiconductor Materials & Equipment
PYPL,0.049528,PayPal,Financials,Transaction & Payment Processing Services
ETSY,0.048329,Etsy,Consumer Discretionary,Broadline Retail
ENPH,0.047827,Enphase,Information Technology,Semiconductor Materials & Equipment
FE,0.04247,FirstEnergy,Utilities,Electric Utilities
TER,0.038997,Teradyne,Information Technology,Semiconductor Materials & Equipment


In [54]:
#listing the top 10 stocks with the lowest return
SPX_returns = SPX_returns.sort_values(by='Return', ascending=True)
print('Top 10 stocks with the lowest return in the S&P 500 of ', today_str, ':', sep='')
SPX_returns.head(10)

Top 10 stocks with the lowest return in the S&P 500 of 2024-02-10:


,Return,Security,Sector,Industry
Symbol,,,,
EXPE,-0.177839,Expedia Group,Consumer Discretionary,"Hotels, Resorts & Cruise Lines"
TTWO,-0.086616,Take-Two Interactive,Communication Services,Interactive Home Entertainment
MRNA,-0.066731,Moderna,Health Care,Biotechnology
NCLH,-0.061714,Norwegian Cruise Line Holdings,Consumer Discretionary,"Hotels, Resorts & Cruise Lines"
DXCM,-0.051791,Dexcom,Health Care,Health Care Equipment
MTD,-0.041193,Mettler Toledo,Health Care,Life Sciences Tools & Services
ILMN,-0.038303,Illumina,Health Care,Life Sciences Tools & Services
PEAK,-0.036131,Healthpeak,Real Estate,Health Care REITs
PEP,-0.035548,PepsiCo,Consumer Staples,Soft Drinks & Non-alcoholic Beverages
